In [7]:
from bs4 import BeautifulSoup 
from selenium import webdriver
from urllib.request import Request, urlopen
import pandas as pd
import json
import sys
import time
from math import*

# PARTIE 1 : COLLECTE DE DONNEES SUR SCIENCEDIRECT

## Comment utiliser le script ? 

### 1) Faire une requête sur ScienceDirect avec votre clé de recherche

### 2) Modifier le nombre de résultat par page par 100
![Result page](images/results_per_page.png)

### 3) Copier url de l'api de la première page de recherche dans first_url 

Pour accéder à l'api : 
- Effectuer un clique droit sur la page et sélectionner 'Inspect'
- Aller dans le menu ouvert à droite et cliquer sur l'onglet 'Network' 
- Rafraichisser la page (F5)
- Effectuer un double clique dans Name sur 'api?talk=...' pour visualiser l'api et copier le lien dans la variable first_url ou copier le lien request url

![tutorial api](images/api_sciencedirect.png)

### 4) Lancer le script de démarrage de collect
Une fenêtre va s'ouvrir avec sélénium, il se peut que l'opération redémarre plusieurs fois.

Le nombre d'article et de page s'affichent au 



In [14]:
# Entrer ici api du lien de la première page de recherche 
first_url = 'https://www.sciencedirect.com/search/api?tak=%28%22multi-label%22%20OR%20%22multilabel%22%29%20AND%20%28%22text%22%20OR%20%E2%80%9Cdocument%E2%80%9D%20OR%20%E2%80%9Cnatural%20language%20process%E2%80%9D%29%20AND%20%28%22class%22%20OR%20%22categorization%22%20OR%20%22tag%22%29&show=100&t=082608fbf78dbf3f72407fceaf94becf53c742c8b4a638c563345288bcadb775&hostname=www.sciencedirect.com'

In [22]:
# Cette fonctionne transforme la clé de recherche en liste de liens de chaque page de résultat
def transform_key_into_api_url(key : str) -> str :
    key = key.replace(' ','%20')
    key = key.replace('(','%28')
    key = key.replace(')','%29')
    key = key.replace('"','%22')
    url = 'https://www.sciencedirect.com/search/api?qs=' + key + '&show=100'
    return url 

#Récupère le code JSON de l'API de ScienceDirect et le transforme un dictionnaraire exploitable
def API_sciencedirect(url : str) -> dict :
    try : 
        navigator = "firefox" # Changer de navigateur ici
        if navigator == "firefox" : browser = webdriver.Firefox()
        elif navigator == "chrome": browser = webdriver.Chrome()
        elif navigator == "ie": browser = webdriver.Ie()
        else : raise Exception("unable to interpret the navigator")
        browser.get(url)
        text = browser.page_source
        text = text.replace('<html platform="linux" class="theme-light" dir="ltr"><head><meta http-equiv="Content-Security-Policy" content="default-src \'none\' ; script-src resource:; "><link rel="stylesheet" type="text/css" href="chrome://devtools-jsonview-styles/content/main.css"><script type="text/javascript" charset="utf-8" async="" data-requirecontext="_" data-requiremodule="viewer-config" src="resource://devtools-client-jsonview/viewer-config.js"></script></head><body><div id="content"><div id="json">', '')
        text = text.replace('</div></div><script src="resource://devtools-client-jsonview/lib/require.js" data-main="resource://devtools-client-jsonview/viewer-config.js"></script></body></html>','')
        text = json.loads(text) 
        browser.close()
        return text  
    except Exception as e :
        print('erreur de chargement de la page... opération recommencée') 
        time.sleep(10)
        browser.close()
        API_sciencedirect(url)
        
# Compte et retourne le nombre de page de résultat de recherche 
def number_page_counter(url : str) -> int :
    try : 
        source_code = API_sciencedirect(url)
        number_article = source_code['resultsFound']
        number_page = ceil(int(number_article)/100)
        print("la recherche contient le nombre suivant d'article :", number_article)
        print("reparti sur le nombre suivant de page :", number_page)
        return number_page
    except Exception as e :
        print("nombre de page non récupéré")
        time.sleep(10)
        number_page_counter(url)
        
# Collecte les liens des pages de recherche 
def find_url_research(first_url : str, number_page : int) -> list :     
    url_list=[first_url]
    for i in range(1,number_page,1):
        i = str(i) # i converti en format string 
        new_url = first_url + '&offset=' + i + '00&t=bff055c9ae3ffce0274c65a3be0bb20e7fe92ad81e7e672aa07499cfeea8b00b'
        url_list.append(new_url)
    return url_list

# collecte les liens des articles
def find_url_articles(url_list : list) -> list :
    article_list_url = []
    number_page = 1
    try :   
        for url in url_list : 
            print('collecte des liens des articles sur la page', number_page)

            while True : 
                source_code = API_sciencedirect(url)
                if source_code is not None :
                    break
            for i in range(100) :
                i = int(i)
                article_url = str(source_code['searchResults'][i]['pdf']['getAccessLink'])
                article_list_url.append(article_url)
            number_page = number_page + 1
        return article_list_url
    except Exception as e :
        return article_list_url
    
# Création d'une dataframe à partir des listes de titre, date, mots clé pour chaque article
def create_dataframe(title_list : list ,date_list : list, keywords_list : list):
       # crée un dictionnaire à partir des listes
        dict = {'Title': title_list, 'Date': date_list , 'Author Keywords': keywords_list}
        #utilisation du dictionnaraire pour créer une dataframe
        df = pd.DataFrame(dict)
        df.to_csv('data/dataSD.csv')  
        print("Collecte de données terminées")

def collect_data(article_list_url : list) : 
    title_list=[]
    keywords_list=[]
    date_list=[]
    i = 1
    for article in article_list_url :
        print("chargement des données de l'article", i) 
        url='https://www.sciencedirect.com' + article
        req = Request(url,headers={'User-Agent': 'Chrome/71.0.3578.98'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage,'html.parser')

        # store keywords in a list 
        try : 
            keywords=soup.find("div",{'class':"Keywords u-font-serif"}).getText(',')
            keywords=keywords.replace('Keywords,','')
            keywords_list.append(keywords)
        except Exception as e : 
            keywords=0
            keywords_list.append(keywords)
            print("Pas de mot clé pour l'article", i)
        # store title in a list 
        title = soup.find("title").getText()
        title = title.replace('- ScienceDirect', '')
        title_list.append(title)
        # store date in a list 
        date = soup.find("meta",  {"name":"citation_publication_date"})
        date_list.append(date["content"])    
        i=i+1
    create_dataframe(title_list ,date_list, keywords_list)

def start_collect_data(first_url):
    while True :
        number_page = number_page_counter(first_url) # récupère le nombre de page de recherche
        if number_page is not None :
            break
    url_list = find_url_research(first_url, number_page) # récupère tout les url des pages de recherche
    article_list_url = find_url_articles(url_list) # créer une liste de tout les articles
    collect_data(article_list_url) # collecte les titres, dates et mots clés des articles et les stocke dans un csv

In [23]:
start_collect_data(first_url)

erreur de chargement de la page... opération recommencée
erreur de chargement de la page... opération recommencée
erreur de chargement de la page... opération recommencée
nombre de page non récupéré
erreur de chargement de la page... opération recommencée
nombre de page non récupéré
la recherche contient le nombre suivant d'article : 71
reparti sur le nombre suivant de page : 1
la recherche contient le nombre suivant d'article : 71
reparti sur le nombre suivant de page : 1
collecte des liens des articles sur la page 1
erreur de chargement de la page... opération recommencée
erreur de chargement de la page... opération recommencée
erreur de chargement de la page... opération recommencée
chargement des données de l'article 1
Pas de mot clé pour l'article 1
chargement des données de l'article 2
chargement des données de l'article 3
chargement des données de l'article 4
chargement des données de l'article 5
chargement des données de l'article 6
chargement des données de l'article 7
chargem

# PARTIE 2 : GENERATION D'UN CSV COMPTANT LES OCCURENCES DE MOTS-CLE

In [3]:
# télécharger le csv
df = pd.read_csv('data/dataSD.csv')
# crée une dataframe avec les "Author Keywords"
df_words = df[['Author Keywords']]

In [4]:
# sépare les valeurs séparées par "," en colonnes 
df_words = df_words['Author Keywords'].str.split(",", expand = True) 

In [5]:
# regroupe toutes les colonnes en une seule colonne
df_words = df_words.stack().reset_index()  

In [6]:
# supprime les colonnes inutilisées 
df_words.drop(columns=['level_0'], inplace=True)
df_words.drop(columns=['level_1'], inplace=True)


In [7]:
# supprime les lignes sans valeur
df_words = df_words[df_words!='0']
df_words = df_words.dropna()


In [8]:
# transformer toutes les lettres en minuscule
df_words = df_words.applymap(lambda s:s.lower() if type(s) == str else s)

In [9]:
# supprimer les espaces à gauche et à droite des mots 
df_words[0] = (df_words[0]).str.lstrip()
df_words[0] = (df_words[0]).str.rstrip()

In [10]:
# compte le nombre d'occurence des keywords
df_words = df_words[0].value_counts()

In [11]:
# télécharge le csv avec le nombre d'occurence par mot
df_words.to_csv('key_words.csv')